In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import dash_table as dt
from dash.dependencies import Input, Output
import plotly.express as px
import base64

import os
import uuid
import pandas as pd
import numpy as np
from functools import lru_cache
from dotenv import load_dotenv
from animal_shelter import AnimalShelter

# Load environment variables
load_dotenv()
username = os.getenv("MONGO_USERNAME")
password = os.getenv("MONGO_PASSWORD")

# Initialize database connection with error handling
try:
    a = AnimalShelter(username, password)
except Exception as e:
    print(f"[ERROR] Failed to connect to database: {e}")
    exit()

uuidOne = uuid.uuid1()

# Initial query cached in dcc.Store
initial_data = list(a.read({"animal_type": "Dog"}))
initial_df = pd.DataFrame.from_records(initial_data)

# Load logo
image_filename = 'Grazioso_Salvare_Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Initialize Dash App
app = JupyterDash('Animal Shelter Dashboard')
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display': 'none'}),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Center(html.B(html.H1('Brandon Barrett - SNHU CS-340'))),
    html.Center(html.P('Grazioso Salvare Animal Shelter Web Application Dashboard')),
    html.Hr(),
    html.Div(dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'WR'},
            {'label': 'Mountain/Wilderness Rescue', 'value': 'MWR'},
            {'label': 'Disaster Rescue/Individual Tracking', 'value': 'DRIT'},
            {'label': 'Reset - Show All', 'value': 'RESET'}
        ],
        value='RESET',
        labelStyle={'display': 'inline-block'}
    )),
    html.Hr(),
    dcc.Store(id='stored-data', data=initial_df.to_dict('records')),  # Cached base data
    dt.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in initial_df.columns],
        data=initial_df.to_dict('records'),
        filter_action="native",
        sort_mode="multi",
        row_selectable="single",
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Div(id='no-data-message', style={'color': 'red', 'textAlign': 'center'}),
    html.Br(),
    html.Hr(),
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

# Utility: Filter logic with caching
@lru_cache(maxsize=10)
def fetch_filtered_data(filter_type):
    if filter_type == 'WR':
        return list(a.read({
            '$and': [
                {'sex_upon_outcome': 'Intact Female'},
                {'breed': {'$in': ['Labrador Retriever Mix', 'Chesa Bay Retr Mix', 'Newfoundland Mix',
                                   'Newfoundland/Labrador Retriever', 'Newfoundland/Australian Cattle Dog',
                                   'Newfoundland/Great Pyrenees']}},
                {'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}}
            ]
        }))
    elif filter_type == 'MWR':
        return list(a.read({
            '$and': [
                {'sex_upon_outcome': 'Intact Male'},
                {'breed': {'$in': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog',
                                   'Rottweiler', 'Siberian Husky']}},
                {'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}}
            ]
        }))
    elif filter_type == 'DRIT':
        return list(a.read({
            '$and': [
                {'sex_upon_outcome': 'Intact Male'},
                {'breed': {'$in': ['Doberman Pinscher', 'German Sheperd', 'Golden Retriever',
                                   'Bloodhound', 'Rottweiler']}},
                {'age_upon_outcome_in_weeks': {'$gte': 20, '$lte': 300}}
            ]
        }))
    else:  # RESET
        return initial_data

# Callback: Filter and store data
@app.callback(
    [Output('stored-data', 'data'),
     Output('datatable-id', 'data'),
     Output('datatable-id', 'columns'),
     Output('no-data-message', 'children')],
    [Input('filter-type', 'value')]
)
def update_data(filter_type):
    data = fetch_filtered_data(filter_type)
    if not data:
        return [], [], [], "No matching records found."
    df = pd.DataFrame(data)
    columns = [{"name": i, "id": i} for i in df.columns]
    return df.to_dict('records'), df.to_dict('records'), columns, ""

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('stored-data', "data")]
)
def update_graphs(viewData):
    if not viewData:
        return [html.P("No data available for graph.")]
    dff = pd.DataFrame(viewData)
    if 'breed' not in dff.columns:
        return [html.P("Missing 'breed' column in data.")]
    names = dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()
    return [dcc.Graph(figure=px.pie(
        data_frame=dff,
        values=values,
        names=names,
        color_discrete_sequence=px.colors.sequential.RdBu,
        width=800,
        height=500
    ))]

@app.callback(
    Output('map-id', "children"),
    [Input('stored-data', "data"),
     Input('datatable-id', 'selected_rows')]
)
def update_map(viewData, selected_rows):
    if not viewData:
        return [html.P("No map data to display.")]
    dff = pd.DataFrame(viewData)
    if selected_rows is None or len(selected_rows) == 0:
        selected_rows = [0]

    try:
        lat = dff.iloc[selected_rows[0], 13]
        lon = dff.iloc[selected_rows[0], 14]
        name = dff.iloc[selected_rows[0], 4]
        description = dff.iloc[selected_rows[0], 9]
        sex = dff.iloc[selected_rows[0], 12]
        age = dff.iloc[selected_rows[0], 15]
    except (IndexError, KeyError):
        return [html.P("Invalid data for map.")]

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(name),
                dl.Popup([
                    html.H4("Animal Name"), html.P(description),
                    html.H4("Sex"), html.P(sex),
                    html.H4("Breed"), html.P(name),
                    html.H4("Age"), html.P(age)
                ])
            ])
        ])
    ]

# Run the app (in Jupyter, use app.run_server(mode='inline'))
if __name__ == '__main__':
    app.run_server(debug=True)

ModuleNotFoundError: No module named 'jupyter_plotly_dash'